In [9]:
import torch
import numpy as np
import pandas as pd

from NeuralProcessClasses_Proteins import *
from architecture_classes import *
from utils import *

import matplotlib.pyplot as plt

In [5]:
train_one_d_datasets = One_D_Datasets(
    kernel = sklearn.gaussian_process.kernels.RBF(length_scale_bounds=(0.1, 2)) 
)

test_one_d_datasets = One_D_Datasets(
    kernel = sklearn.gaussian_process.kernels.RBF(length_scale_bounds=(0.1, 2)) 
)

val_one_d_datasets = One_D_Datasets(
    num_samples = 1000,
    kernel = sklearn.gaussian_process.kernels.RBF(length_scale_bounds=(0.1, 2)) 
)

In [7]:
train_one_d_datasets.get_samples(None, None, None)
test_one_d_datasets.get_samples(None, None, None)
val_one_d_datasets.get_samples(None, None, None)

(array([[-1.98248551, -1.91442858, -1.89392648, ...,  1.87617796,
          1.89508134,  1.89537335],
        [-1.96404742, -1.9552422 , -1.94146547, ...,  1.77239544,
          1.81415122,  1.99234346],
        [-1.99283572, -1.97805046, -1.94541975, ...,  1.92429742,
          1.94527873,  1.97413049],
        ...,
        [-1.99266505, -1.98688572, -1.97616397, ...,  1.94773713,
          1.95055513,  1.97158542],
        [-1.91388798, -1.89624852, -1.83027646, ...,  1.96086001,
          1.97877262,  1.99330903],
        [-1.94366276, -1.91984096, -1.91955743, ...,  1.86985792,
          1.87496219,  1.93971352]]),
 tensor([[[ 1.0173],
          [ 1.0705],
          [ 1.0857],
          ...,
          [-0.9395],
          [-0.9237],
          [-0.9234]],
 
         [[ 1.0406],
          [ 1.0366],
          [ 1.0300],
          ...,
          [-3.8200],
          [-3.8229],
          [-3.7823]],
 
         [[-0.8260],
          [-0.8356],
          [-0.8552],
          ...,
       

In [8]:
for i in range(10):
    X, Y = train_one_d_datasets[i]
    plt.plot(X,Y)

NameError: name 'plt' is not defined

In [ ]:
class AttentiveEncoder(torch.nn.Module):
    def __init__(
        activation = torch.nn.SELU(),
        rate,
        x_dim,
        y_dim,
        projected_dim,
        heads,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.activation = activation
        self.addnorm = AddNorm(projected_dim)
        self.dropout = torch.nn.Dropout(rate)
        
        self.linear_1 = torch.nn.Linear(x_dim + y_dim, projected_dim)

        self.self_mha_1 = MultiHeadedAttention(heads, projected_dim, projected_dim, projected_dim, d_hidden, projected_dim)
        self.self_mha_2 = MultiHeadedAttention(heads, projected_dim, projected_dim, projected_dim, d_hidden, projected_dim)
        
        self.context_projection = torch.nn.Linear(x_dim, projected_dim)
        self.target_projection = torch.nn.Linear(x_dim, projected_dim)

        self.cross_mha  = MultiHeadedAttention(heads, projected_dim, projected_dim, projected_dim, d_hidden, projected_dim)

        self.linear_2 = torch.nn.Linear(projected_dim + x_dim, projected_dim)
        self.linear_3 = torch.nn.Linear(projected_dim + x_dim, projected_dim)
        self.linear_4 = torch.nn.Linear(projected_dim + x_dim, projected_dim)
        self.linear_5 = torch.nn.Linear(projected_dim + x_dim, projected_dim)

    def forward(self, x, y):
        input = torch.concat([x, y], dim=-1)

        x_1 = self.activation(self.linear_1(input))

        x_2, _ = self.self_mha_1(x_1, x_1, x_1)
        x_2    = self.addnorm(x_2, x_1)
        x_2    = self.dropout(x_2)

        x_3, _ = self.self_mha_2(x_2, x_2, x_2)
        x_3    = self.addnorm(x_3, x_2)
        x_3    = self.dropout(x_3)
        return x_3


class AttentiveDecoder(torch.nn.Module):
    